<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install yfinance==0.2.59
!pip install ta

In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Spread%', 'Conviction', 'Cyclical','RSI_14', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%', 'ROE%/PE']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-12-03 16:19:15
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


get_prospects_features
qualified stocks: 290
with latest results: 85
still star stocks: 63


In [7]:
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Spread%', 'Conviction','Cyclical', 'RSI_14', 'RSP', 'FTT%', 'ATH%','Gained%', 'CumlRnk','ROE%/PE', 'Criteria', 'Strategy', 'Category', 'InFolio']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
list_ox40 = list(tmp_df[tmp_df['MBQ'].str.contains('OX40', na=False)]['Symbol'].values)
list_ox40n = list(tmp_df[tmp_df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list(set(list_x40 + list_x5k + list_hc)- set(list_ox40 + list_ox40n))))

In [8]:
df_tmp = df_prospects_features.copy()

# value ranks (ascending better for deviation from PE and 200DMA)
df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_ROE%/PE'] = df_tmp['ROE%/PE'].rank(method='max', ascending=False)

# growth ranks
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)

# quality ranks
df_tmp['Rnk_FTT%'] = df_tmp['FTT%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)

# optional Momentum ranks
df_tmp['Rnk_RSI_14'] = df_tmp['RSI_14'].rank(method='max', ascending=False)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)

# category scores
df_tmp['Rank_Value'] = df_tmp[['Rnk_Dev%_200', 'Rnk_Dev%_PE', 'Rnk_ROE%/PE']].sum(axis=1)
df_tmp['Rank_Growth'] = df_tmp[['Rnk_Sales_Grwth%', 'Rnk_Profit_Grwth%']].sum(axis=1)
df_tmp['Rank_Quality'] = df_tmp[['Rnk_FTT%', 'Rnk_MCap', 'Rnk_ROE%', 'Rnk_CFO_2_EBITDA%']].sum(axis=1)
df_tmp['Rank_Momentum'] = df_tmp[['Rnk_Gained%', 'Rnk_RSI_14']].sum(axis=1)

# final cumulative score (you can weigh categories if needed)
df_tmp['Ovrl_Rank'] = df_tmp[['Rank_Value', 'Rank_Growth', 'Rank_Quality', 'Rank_Momentum']].sum(axis=1)

# category-wise rank
df_tmp['Cat_Rank'] = df_tmp.groupby('Conviction')['Ovrl_Rank'].rank(ascending=True, method='min')
priority_map = {'X-LC': 0,'H-LC': 1,'X-MC': 2,'X-SC': 3,'M-LC': 4,'H-MC': 5,'H-SC': 6,'L-LC': 7,'M-MC': 8,'M-SC': 9,'L-MC': 10,'L-SC': 11}
df_tmp['Conviction_Priority'] = df_tmp['Conviction'].map(priority_map)
df_tmp = df_tmp.sort_values(by=['Conviction_Priority', 'Cat_Rank']).reset_index(drop=True)

# final rank
df_tmp['CumlRnk'] = df_tmp.index + 1

# drop intermediate rank columns (optional)
drop_cols = [col for col in df_tmp.columns if col.startswith('Rnk_')]
df_tmp.drop(columns=drop_cols, inplace=True)

# reassign to main dataframe
df_prospects_features = df_tmp[cols]

# display
data_table.DataTable(df_prospects_features.sort_values(by='CumlRnk'), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4389.97,-2.42,-24.60,6.73,X-LC,NC,64.0,88.24,38.05,38.05,10.51,1,2.2,X40,ATH,IT,1.0
1,INFY,2275.00,3.73,-16.37,2.71,X-LC,NC,64.0,90.14,44.11,23.03,16.37,2,1.2,X40,BTT,IT,1.0
2,VBL,671.64,-1.55,-14.99,5.39,X-LC,NC,58.0,75.26,40.66,40.66,9.82,3,0.4,X40N,ATH,FMCG,1.0
3,TMPV,600.00,-8.03,-24.30,3.13,X-LC,DC,32.0,4.84,68.14,16.86,1.25,4,4.5,XY24,NTT,AUTO,1.0
4,ITC,452.00,-2.55,-41.56,1.51,X-LC,NC,40.0,53.29,12.86,26.17,3.30,5,1.9,X40,NTT,FMCG,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,SPORTKING,155.85,-10.93,NaN,4.50,NaN,DC,34.0,20.76,71.55,71.55,27.89,286,NaN,AR,ATH,TEXTILES,NaN
286,TASTYBITE,20384.08,-13.39,NaN,11.01,NaN,NC,31.0,23.53,147.08,147.08,5.99,287,NaN,XR,ATH,FMCG,NaN
287,V2RETAIL,2495.60,23.26,NaN,24.31,NaN,DC,49.0,31.14,6.62,6.62,77.39,288,NaN,AR,ATH,TEXTILES,NaN
288,VEEDOL,2988.63,1.09,NaN,8.53,NaN,SC,36.0,30.80,83.40,83.40,23.20,289,NaN,XR,ATH,CHEMICALS,NaN


In [9]:
# most interesting sectors
df_tmp = df_prospects_features.groupby('Category').agg(
    Count=('CumlRnk', 'count'),
    Total=('CumlRnk', 'sum'),
    Average=('CumlRnk', 'mean')
).reset_index()
df_tmp['Average'] = round(df_tmp['Average'],2)
df_tmp = df_tmp[df_tmp['Count'] > 3]
df_tmp.sort_values(by='Average')

,Category,Count,Total,Average
7,CEMENT,4,337,84.25
16,FMCG,19,1645,86.58
27,METALS,5,487,97.40
3,AUTO,12,1180,98.33
23,INSURANCE,8,806,100.75
33,PHARMA,18,2040,113.33
24,IT,18,2227,123.72
12,ELECTRICAL,9,1170,130.00
20,HEALTHCARE,8,1075,134.38
21,HOTELS,4,547,136.75


In [10]:
# module to print date timestamp
def read_csv(csv_file_name):
  csv_file_path = os.path.join(project_path, 'data', csv_file_name)
  df = pd.read_csv(csv_file_path)
  return df

mypps_df = read_csv('myProspectsScrips.csv')

# Suppose this is your updated DataFrame with new cuml_rnk values
cols1 = ['Symbol', 'CumlRnk']
df_tmp = df_prospects_features[cols1]

# Merge the two DataFrames on 'symbol', keeping the CSV structure
mypps_df = mypps_df.merge(df_tmp[['Symbol', 'CumlRnk']], on='Symbol', how='left', suffixes=('', '_new'))

# Replace 'cuml_rnk' with the new value only if it exists in df
mypps_df['CumlRnk'] = mypps_df['CumlRnk_new'].combine_first(mypps_df['CumlRnk'])

# Drop the helper column
mypps_df.drop(columns=['CumlRnk_new'], inplace=True)

# Save back to CSV
#mypps_df.to_csv(project_path+'/data/myProspectsScrips.csv', index=False)

In [11]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]
df_tmp = df_tmp[(df_tmp['FTT%']>20) & (df_tmp['Dev%_PE']<0)]
data_table.DataTable(df_tmp.sort_values(by = 'CumlRnk', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
0,TCS,4389.97,-2.42,-24.60,6.73,X-LC,NC,64.0,88.24,38.05,38.05,10.51,1,2.2,X40,ATH,IT,1.0
1,INFY,2275.00,3.73,-16.37,2.71,X-LC,NC,64.0,90.14,44.11,23.03,16.37,2,1.2,X40,BTT,IT,1.0
2,VBL,671.64,-1.55,-14.99,5.39,X-LC,NC,58.0,75.26,40.66,40.66,9.82,3,0.4,X40N,ATH,FMCG,1.0
3,TMPV,600.00,-8.03,-24.30,3.13,X-LC,DC,32.0,4.84,68.14,16.86,1.25,4,4.5,XY24,NTT,AUTO,1.0
15,DMART,5391.45,-6.20,-19.86,4.35,X-LC,NC,29.0,34.43,37.90,37.90,14.86,16,0.1,X40N,ATH,FMCG,1.0
19,BAJAJFINSV,2758.00,2.51,-8.00,3.65,X-LC,NC,46.0,65.05,34.80,56.41,31.28,20,0.4,X40,BTT,FINANCE,NaN
31,BAJAJHLDNG,14451.52,-11.72,-2.00,6.64,X-LC,NC,39.0,25.61,28.78,28.78,9.55,32,0.7,X40,ATH,FINANCE,NaN
35,JIOFIN,387.00,5.24,-4.82,6.96,X-LC,NC,43.0,62.28,28.57,28.69,50.04,36,0.0,XY24,BTT,FINANCE,1.0
38,DLF,885.00,-4.97,-26.13,0.86,H-LC,DC,30.0,17.30,25.00,44.66,17.29,39,0.3,AR,NTT,REALTY,NaN
54,PAGEIND,51605.08,-14.50,-33.02,10.49,X-MC,DC,29.0,20.42,38.57,38.57,0.00,55,0.9,X40,ATH,APPARELS,1.0


In [12]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
204,EASEMYTRIP,26.40,-18.00,-16.85,23.43,M-SC,DC,61.0,84.95,221.95,315.98,15.33,205,0.9,XY24,NTT,TRAVEL,1.0
169,BANDHANBNK,400.00,-9.40,38.65,6.38,H-SC,NC,33.0,28.03,172.52,381.37,13.67,170,0.6,XY24,NTT,BANKS,1.0
160,REPCOHOME,880.00,4.21,-56.78,8.90,H-SC,NC,41.0,17.99,119.92,107.06,30.84,161,2.5,XY24,NTT,FINANCE,1.0
234,GREENPANEL,537.00,-9.32,167.44,6.79,M-SC,NC,30.0,6.92,119.33,144.94,12.39,235,0.3,XY24,NTT,MISC,1.0
81,ACC,3906.00,-2.06,-54.57,1.45,X-SC,DC,46.0,66.09,112.26,50.04,3.23,82,1.3,XY24,BTT,CEMENT,1.0
240,SFL,1287.00,-11.17,18.85,7.90,M-SC,NC,37.0,4.15,111.50,224.11,2.60,241,0.0,XY24,NTT,MISC,1.0
82,QUESS,424.00,-39.88,-54.27,37.42,X-SC,NC,37.0,8.48,104.41,464.66,0.00,83,1.3,XY24,NTT,MISC,1.0
203,DIGITIDE,267.99,-29.34,NaN,24.82,M-SC,NC,33.0,5.19,96.52,96.52,0.26,204,NaN,XY24,ATH,IT,NaN
58,AWL,485.00,-5.87,-62.10,1.96,X-MC,NC,28.0,13.49,95.92,239.73,5.01,59,0.5,XY24,NTT,FMCG,1.0
173,TITAGARH,1548.00,-6.45,-0.43,3.00,H-SC,DC,29.0,7.27,94.91,133.77,17.77,174,0.3,XY24,NTT,ENGINEERING,1.0


In [13]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
90,RELAXO,1176.00,-7.69,-43.96,5.01,X-SC,SC,35.0,30.10,189.55,248.27,2.21,91,0.1,X40N,NTT,FOOTWEAR,1.0
85,TEAMLEASE,3487.00,-14.12,114.11,10.17,X-SC,NC,36.0,40.83,116.32,227.59,0.00,86,0.5,X40N,NTT,MISC,NaN
92,BATAINDIA,2096.00,-17.45,6.43,14.19,X-SC,SC,26.0,17.65,114.44,115.25,1.32,93,0.2,X40,NTT,FOOTWEAR,1.0
81,ACC,3906.00,-2.06,-54.57,1.45,X-SC,DC,46.0,66.09,112.26,50.04,3.23,82,1.3,XY24,BTT,CEMENT,1.0
82,QUESS,424.00,-39.88,-54.27,37.42,X-SC,NC,37.0,8.48,104.41,464.66,0.00,83,1.3,XY24,NTT,MISC,1.0
58,AWL,485.00,-5.87,-62.10,1.96,X-MC,NC,28.0,13.49,95.92,239.73,5.01,59,0.5,XY24,NTT,FMCG,1.0
61,BAJAJHFL,181.50,-16.41,-26.32,8.85,X-MC,NC,13.0,9.69,87.17,87.17,0.00,62,0.4,X40N,ATH,FINANCE,1.0
62,COLPAL,3726.84,-10.21,-4.98,7.32,X-MC,NC,30.0,48.44,77.31,77.31,0.00,63,1.9,XY25,ATH,FMCG,1.0
84,SANOFI,7552.00,-17.92,34.34,15.88,X-SC,NC,34.0,29.76,71.66,121.32,0.93,85,1.7,X40,NTT,PHARMA,NaN
87,HONAUT,58357.33,-4.10,-26.21,2.99,X-SC,SC,44.0,48.79,66.78,66.78,7.46,88,0.2,X40N,ATH,ELECTRICAL,1.0


In [14]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
153,TANLA,1943.92,-4.42,-36.34,8.62,H-SC,NC,33.0,12.11,247.66,247.66,35.17,154,1.6,AR,ATH,IT,1.0
145,VALIANTORG,838.00,-25.12,-255.96,19.56,H-SC,SC,32.0,3.81,228.18,624.75,11.29,146,0.0,XR,NTT,CHEMICALS,1.0
137,ROUTE,2227.21,-21.13,-56.81,23.77,H-SC,NC,54.0,59.17,216.23,216.23,9.70,138,1.0,SR,ATH,IT,1.0
147,HAPPSTMNDS,1480.71,-14.87,-26.17,14.24,H-SC,NC,44.0,58.48,197.84,197.84,2.09,148,0.3,AR,ATH,IT,1.0
143,ALKYLAMINE,4546.37,-13.22,-24.40,8.88,H-SC,SC,23.0,7.96,174.14,174.14,6.22,144,0.3,SR,ATH,CHEMICALS,1.0
169,BANDHANBNK,400.00,-9.40,38.65,6.38,H-SC,NC,33.0,28.03,172.52,381.37,13.67,170,0.6,XY24,NTT,BANKS,1.0
171,HATHWAY,31.40,-7.50,802.17,6.21,H-SC,DC,37.0,37.37,142.47,471.97,3.93,172,0.1,XR,NTT,ENTERTAINMENT,1.0
141,CLEAN,2137.00,-25.93,-43.61,23.24,H-SC,SC,31.0,10.03,138.64,193.19,0.00,142,0.7,XR,NTT,CHEMICALS,NaN
139,SONATSOFTW,837.75,-3.33,-13.69,3.12,H-SC,NC,50.0,66.44,129.87,129.87,23.15,140,1.1,MGC,ATH,IT,NaN
156,VAIBHAVGBL,521.00,0.64,52.62,7.36,H-SC,DC,39.0,4.50,122.18,290.72,19.10,157,0.4,XR,NTT,JEWELLERY,1.0


In [15]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
92,BATAINDIA,2096.00,-17.45,6.43,14.19,X-SC,SC,26.0,17.65,114.44,115.25,1.32,93,0.2,X40,NTT,FOOTWEAR,1.0
84,SANOFI,7552.00,-17.92,34.34,15.88,X-SC,NC,34.0,29.76,71.66,121.32,0.93,85,1.7,X40,NTT,PHARMA,NaN
66,HAVELLS,2069.16,-6.54,-13.63,4.93,X-MC,SC,38.0,42.91,45.65,45.65,1.35,67,0.3,X40,ATH,ELECTRICAL,1.0
1,INFY,2275.00,3.73,-16.37,2.71,X-LC,NC,64.0,90.14,44.11,23.03,16.37,2,1.2,X40,BTT,IT,1.0
57,PGHH,17907.65,-6.48,-33.45,4.52,X-MC,NC,29.0,37.02,41.69,41.69,0.00,58,1.5,X40,ATH,FMCG,1.0
54,PAGEIND,51605.08,-14.50,-33.02,10.49,X-MC,DC,29.0,20.42,38.57,38.57,0.00,55,0.9,X40,ATH,APPARELS,1.0
0,TCS,4389.97,-2.42,-24.60,6.73,X-LC,NC,64.0,88.24,38.05,38.05,10.51,1,2.2,X40,ATH,IT,1.0
56,GLAXO,3466.20,-10.99,-18.74,11.18,X-MC,NC,44.0,32.01,37.61,37.61,30.44,57,1.0,X40,ATH,PHARMA,1.0
13,BAJAJ-AUTO,12353.32,7.26,32.46,6.55,X-LC,DC,52.0,76.47,37.25,37.25,26.30,14,0.8,X40,ATH,AUTO,NaN
19,BAJAJFINSV,2758.00,2.51,-8.00,3.65,X-LC,NC,46.0,65.05,34.80,56.41,31.28,20,0.4,X40,BTT,FINANCE,NaN


In [16]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
90,RELAXO,1176.00,-7.69,-43.96,5.01,X-SC,SC,35.0,30.10,189.55,248.27,2.21,91,0.1,X40N,NTT,FOOTWEAR,1.0
85,TEAMLEASE,3487.00,-14.12,114.11,10.17,X-SC,NC,36.0,40.83,116.32,227.59,0.00,86,0.5,X40N,NTT,MISC,NaN
61,BAJAJHFL,181.50,-16.41,-26.32,8.85,X-MC,NC,13.0,9.69,87.17,87.17,0.00,62,0.4,X40N,ATH,FINANCE,1.0
87,HONAUT,58357.33,-4.10,-26.21,2.99,X-SC,SC,44.0,48.79,66.78,66.78,7.46,88,0.2,X40N,ATH,ELECTRICAL,1.0
96,KAJARIACER,1526.69,1.19,-6.26,10.02,X-SC,SC,32.0,14.88,42.87,42.87,38.44,97,0.2,X40N,ATH,CERAMICS,NaN
2,VBL,671.64,-1.55,-14.99,5.39,X-LC,NC,58.0,75.26,40.66,40.66,9.82,3,0.4,X40N,ATH,FMCG,1.0
15,DMART,5391.45,-6.20,-19.86,4.35,X-LC,NC,29.0,34.43,37.90,37.90,14.86,16,0.1,X40N,ATH,FMCG,1.0
55,DIXON,18931.72,-9.92,-54.61,5.88,X-MC,NC,28.0,15.22,35.05,35.05,11.69,56,0.6,X40N,ATH,IT,1.0
16,GODREJCP,1482.76,-4.54,7.56,3.27,X-LC,NC,44.0,48.10,32.31,32.31,14.59,17,0.2,X40N,ATH,FMCG,NaN
77,UBL,2227.43,-11.49,1.32,9.15,X-MC,NC,38.0,33.56,30.93,30.93,1.81,78,0.1,X40N,ATH,FMCG,NaN


In [17]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Spread%,Conviction,Cyclical,RSI_14,RSP,FTT%,ATH%,Gained%,CumlRnk,ROE%/PE,Criteria,Strategy,Category,InFolio
272,BAJAJHIND,442.94,-10.09,NaN,5.46,NaN,DC,33.0,8.48,2139.33,2139.33,12.96,273,NaN,XR,ATH,SUGAR,NaN
260,CENTRALBK,187.19,-1.89,-47.48,0.87,L-SC,NC,41.0,29.07,402.12,402.12,11.78,261,1.6,XR,ATH,BANKS,NaN
259,ALOKINDS,79.81,-8.44,NaN,5.52,L-SC,DC,34.0,43.60,384.28,384.28,10.75,260,NaN,AR,ATH,TEXTILES,NaN
153,TANLA,1943.92,-4.42,-36.34,8.62,H-SC,NC,33.0,12.11,247.66,247.66,35.17,154,1.6,AR,ATH,IT,1.0
145,VALIANTORG,838.00,-25.12,-255.96,19.56,H-SC,SC,32.0,3.81,228.18,624.75,11.29,146,0.0,XR,NTT,CHEMICALS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,MARUTI,16424.00,19.36,-2.48,18.83,X-LC,DC,55.0,82.01,2.13,2.13,51.41,31,0.5,XR,ATH,AUTO,NaN
48,HEROMOTOCO,6295.50,35.65,18.35,27.49,H-LC,DC,73.0,95.50,1.35,1.35,80.51,49,0.9,AR,ATH,AUTO,NaN
110,MUTHOOTFIN,3811.10,41.11,68.27,34.88,M-LC,NC,69.0,99.31,1.27,1.27,97.10,111,0.8,XR,ATH,FINANCE,NaN
198,UPL,737.00,10.40,54.30,11.90,M-MC,SC,52.0,78.20,-0.81,9.13,50.06,199,0.1,AR,NTT,CHEMICALS,NaN


In [18]:
# conviction-wise prospects count
df_prospects_features['Conviction'].value_counts()

,count
Conviction,
M-SC,51
H-SC,42
X-LC,36
X-MC,27
M-LC,19
H-LC,18
X-SC,17
H-MC,17
M-MC,17
